# old

In [ ]:
import torch

class CustomTensor(torch.Tensor):
    __slots__ = ('_is_leaf', '_node_id','_custom_requires_grad', '_backward')

    def __new__(cls, data, *,_custom_requires_grad=False, is_leaf=False, graph=None,**kwargs):
        # Construct the real tensor
        if not isinstance(data, torch.Tensor):
            data = torch.tensor(data, **kwargs)

        instance = torch.Tensor._make_subclass(cls, data)
        instance.requires_grad_(False)
        instance._custom_requires_grad = _custom_requires_grad
        instance._is_leaf = is_leaf
        instance._node_id = None
        instance._backward = lambda:None
        if is_leaf and _custom_requires_grad and graph:
            instance._node_id = graph.add_tensor_graph(instance)


        
        
        return instance
    
    def _wrap(self,*,_custom_requires_grad,is_leaf):
        self._custom_requires_grad = _custom_requires_grad
        self._is_leaf = is_leaf
        self._node_id = None
        self._backward = lambda:None
        


In [11]:
a=CustomTensor([1,2,3],_custom_requires_grad=True,is_leaf=True)
b=CustomTensor([1,2,3],_custom_requires_grad=True,is_leaf=True)

In [12]:
c=a+b

In [13]:
type(c)

__main__.CustomTensor

In [14]:
c._wrap(_custom_requires_grad=True,is_leaf=False)

In [16]:
c._is_leaf

False

In [22]:
from torch.utils._python_dispatch import TorchDispatchMode
import weakref
import rustworkx as rx

In [20]:
class AutogradMode(TorchDispatchMode):
    def __init__(self,check_for_cycles=True, auto_cleanup=True):
        self.graph = rx.PyDiGraph()
        self.intermediate_tensors = dict()
        self._check_cycles = check_for_cycles
        self._auto_cleanup = auto_cleanup

    
    def __enter__(self):
        return super().__enter__()
    
    def __exit__(self, *args):
        if self._check_cycles:
            if not self.check_cycle():
                raise RuntimeError("Cycle detected in autograd graph on context exit.")
        if self._auto_cleanup:
            self.intermediate_tensors.clear()
            self.graph.clear()  # Clears all nodes and edges
        return super().__exit__(*args)
    
    def __torch_dispatch__(self, func, types, args=(), kwargs=None):
        # 1) unwrap TrackingTensor → raw torch.Tensor
        if kwargs is None:
            kwargs = {}
        result = func(*args, **kwargs)
        result._wrap(_custom_requires_grad=True,_is_leaf=False)
        # TODO write poper LOGICAL FLOW here
        return result
    def add_tensor_graph(self, tensor):
        requires_grad = tensor._custom_requires_grad
        if not requires_grad:
            raise ValueError("Tensor with require grad False cannot to be added to the graph.")

        tensor_index = self.graph.add_node(weakref.ref(tensor))
        tensor._node_id  = tensor_index

    def add_non_leaf_tensor_references(self, tensor):
        requires_grad = tensor._custom_requires_grad
        is_leaf = tensor._is_leaf
        node_id = tensor._node_id

        if not requires_grad or is_leaf:
            raise ValueError("Tensor must be a non leaf tensor.")

        if node_id in self.intermediate_tensors:
            raise ValueError("Tensor reference to persist in memory already exists.")

        self.intermediate_tensors[node_id] = tensor

    def add_edge(self, node_from, node_to, weight=None):
        if not isinstance(node_from, int) or not isinstance(node_to, int):
            raise TypeError("Node indices must be integers.")

        graph = self.graph
        if not graph.has_node(node_from) or not graph.has_node(node_to):
            raise ValueError("Both nodes must exist in the graph before adding an edge.")

        graph.add_edge(node_from, node_to, weight)
    
    def check_cycle(self):
        return rx.is_directed_acyclic_graph(self.graph)
    
    def reverse_toposort(self):
       
        if not self.check_cycle():
            raise RuntimeError("Cannot perform topological sort on a graph with cycles.")
        graph = self.graph
        node_indexes = rx.topological_sort(graph)
        return [graph[node_index] for node_index in reversed(node_indexes)]
    
    def delete_node(self, node_index):
        if not isinstance(node_index, int):
            raise TypeError("Node index must be an integer.")

        graph = self.graph
        if not graph.has_node(node_index):
            raise ValueError(f"Node index {node_index} does not exist in the graph.")

        graph.remove_node(node_index)

    def delete_edge(self, node_from, node_to):
        if not isinstance(node_from, int) or not isinstance(node_to, int):
            raise TypeError("Node indices must be integers.")

        graph = self.graph
        if not graph.has_edge(node_from, node_to):
            raise ValueError(f"Edge ({node_from}, {node_to}) does not exist in the graph.")

        graph.remove_edge(node_from, node_to)

    def del_non_leaf_tensor_reference(self, tensor_node_id):
        try:
            del self.intermediate_tensors[tensor_node_id]
        except KeyError:
            raise KeyError(f"No tensor reference found for node ID {tensor_node_id}")

    def __repr__(self):
        graph = self.graph
        return f"CustomAutogradGraph(nodes={graph.num_nodes()}, edges={graph.num_edges()})"
    



In [ ]:
type(c)

__main__.CustomTensor

In [23]:
gradmode=AutogradMode()

In [27]:
c

CustomTensor([2, 4, 6])

In [26]:
with gradmode as g:
    a=CustomTensor([1,2,3],_custom_requires_grad=True,is_leaf=True,graph=g.graph)
    b=CustomTensor([1,2,3],_custom_requires_grad=True,is_leaf=True,graph=g.graph)
    c=a+b
    # c._wrap(_custom_requires_grad=True,is_leaf=False)
    # c._is_leaf
    # c._node_id

AttributeError: 'Tensor' object has no attribute '_wrap'

# New

In [ ]:
import torch
import weakref
import numbers

In [4]:
class Operations:
    @torch.jit.script
    def add_tensor_and_scalar(tensor: torch.Tensor, scaler: float) -> torch.Tensor:
        return tensor + scaler
    
    @torch.jit.script
    def add_tensor_and_tensor(tensor1: torch.Tensor, tensor2: torch.Tensor) -> torch.Tensor:
        return tensor1 + tensor2
    

In [ ]:
import torch

class CustomTensor:
    __slots__ = ('tensor','_is_leaf', '_node_id','_custom_requires_grad', '_backward','graph')

    def __init__(self, tensor, *,_custom_requires_grad=False, is_leaf=False,device=None,dtype=None,graph=None):
        # Construct the real tensor
        if not isinstance(tensor, torch.Tensor):
            tensor = torch.tensor(tensor,requires_grad=False, dtype=dtype,device=device)
        elif isinstance(tensor,CustomTensor):
            return tensor
        self.tensor = tensor
        self._custom_requires_grad = _custom_requires_grad
        self._is_leaf = is_leaf
        self._node_id = None
        self._backward = lambda:None
        self.graph = None
        if  _custom_requires_grad:
            self.graph = weakref.ref(graph)
            self._node_id = self.graph.add_tensor_graph(self)
            if not is_leaf:
                self.graph.add_non_leaf_tensor_references(self)
    
    def __add__(self,other):
        
        if isinstance(other, numbers.Number):
            result = Operations.add_tensor_and_scalar(self.tensor,other)
            requires_grad = self._custom_requires_grad
            if requires_grad:
                graph = self.graph()
                result = CustomTensor(result,_custom_requires_grad=requires_grad,is_leaf=False,graph=graph)
                graph.add_edge(self._node_id, result._node_id)
                return result
            else:
                return CustomTensor(result,_custom_requires_grad=requires_grad,is_leaf=True)
                





    

    # def __new__(cls, data, *,_custom_requires_grad=False, is_leaf=False, graph=None,**kwargs):
    #     # Construct the real tensor
    #     if not isinstance(data, torch.Tensor):
    #         data = torch.tensor(data, **kwargs)

    #     instance = torch.Tensor._make_subclass(cls, data)
    #     instance.requires_grad_(False)
    #     instance._custom_requires_grad = _custom_requires_grad
    #     instance._is_leaf = is_leaf
    #     instance._node_id = None
    #     instance._backward = lambda:None
    #     if is_leaf and _custom_requires_grad and graph:
    #         instance._node_id = graph.add_tensor_graph(instance)


        
        
    #     return instance
    
    # def _wrap(self,*,_custom_requires_grad,is_leaf):
    #     self._custom_requires_grad = _custom_requires_grad
    #     self._is_leaf = is_leaf
    #     self._node_id = None
    #     self._backward = lambda:None
        


In [1]:
import numbers

In [ ]:
class Operations:
    @torch.jit.script
    def add_tensor_and_scalar(tensor: torch.Tensor, scaler: float) -> torch.Tensor:
        return tensor + scaler
    
    @torch.jit.script
    def add_tensor_and_tensor(tensor1: torch.Tensor, tensor2: torch.Tensor) -> torch.Tensor:
        return tensor1 + tensor2
    


In [2]:
import torch

@torch.jit.script
def add_tensor_and_scalar(tensor: torch.Tensor, scaler: float) -> torch.Tensor:
    return tensor + scaler

In [3]:
import torch

# Create a tensor
tensor = torch.tensor([1, 2, 3, 4, 5])

# Add a scalar to the tensor
result = add_tensor_and_scalar(tensor, 2.5)

print(result)  # Expected output: tensor([3.5000, 4.5000, 5.5000, 6.5000, 7.5000])

# Test with a different scalar
result = add_tensor_and_scalar(tensor, 10)

print(result)  # Expected output: tensor([11, 12, 13, 14, 15])

# Test with a tensor of different shape
tensor = torch.tensor([[1, 2], [3, 4]])
result = add_tensor_and_scalar(tensor, 2)

print(result)  # Expected output: tensor([[3, 4], [5, 6]])

# Test with a non-numeric scalar
try:
    result = add_tensor_and_scalar(tensor, "hello")
except TypeError as e:
    print(e)  # Expected output: TypeError: unsupported operand type(s) for +: 'Tensor' and 'str'

tensor([3.5000, 4.5000, 5.5000, 6.5000, 7.5000])
tensor([11., 12., 13., 14., 15.])
tensor([[3., 4.],
        [5., 6.]])


RuntimeError: add_tensor_and_scalar() Expected a value of type 'float' for argument 'scaler' but instead found type 'str'.
Position: 1
Value: 'hello'
Declaration: add_tensor_and_scalar(Tensor tensor, float scaler) -> Tensor
Cast error details: Unable to cast Python instance of type <class 'str'> to C++ type '?' (#define PYBIND11_DETAILED_ERROR_MESSAGES or compile in debug mode for details)